# PES 2020 Players Dataset Collection (from http://pesdb.net/pes2020) 

## Motivation:
    
    As a soccer fan, I am crazy for soccer video games (especially FIFA and PES). I spent a lot of time on playing these games to kill time and practice the new strategies virtually. As I went to college, sports analytics became one of the areas that I am interested in. As I felt more determined to go deep into data, I would like to strat a project considering the rate of each player in a game and see how the factors determine the overall ratings. 

## Major Techniques:
    
    - Pandas
    - BeautifulSoups
    - TBD

## Data Collection:
   **The first step is to create a dataframe that hold all the data.**
  

In [19]:
import pandas as pd

# This is the major data framework.
# You ONLY need to run this following code for the first time

df = pd.DataFrame(columns=('Name','Height','Weight','Age','Position','Offensive Awareness','Ball Control','Dribbling','Tight Possession',
                           'Low Pass','Lofted Pass','Finishing','Heading','Place Kicking','Curl','Speed','Acceleration','Kicking Power',
                           'Jump','Physical Contact','Balance','Stamina','Defensive Awareness','Ball Winning','Aggression','GK Awareness',
                           'GK Catching','GK Clearing','GK Reflexes','GK Reach','Weak Foot Usage','Weak Foot Accuracy','Form',
                           'Injury Resistance','Overall Rating'))

df.to_excel(r"C:\Users\qiuwk\Google Drive\Projects\PES.xlsx") # Change the path to where you want it to be

   **_The following program is to create a list of players' ID so that we can access each player's profile._**
   
   **Note**: The website prevents me from requesting too often (I think that's a protection for web-scraping), so I can set a  "sleeping" time of about 2 minutes for every ten visites to the website. But I defintely want to see a better solution :)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time

id_list = []
count = 0

base_url = 'http://pesdb.net/pes2020'
 
for i in range(471):
    url = base_url + '/?page=' + str(i+1)
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features = 'lxml')
    id_source = soup.find_all("a", href= re.compile("^\./\?id=\d+"))
    for j in id_source:
        id_list.append(j.get('href'))
    count += 1
    print(i)
    if count == 10:
        time.sleep(115)
        count = 0

# Lastly, I will save my ID list to a local spreadsheet.
df_2 = pd.DataFrame(data = id_list, columns = 'ID')
df_2.to_excel(r"C:\Users\qiuwk\Google Drive\Projects\PES_tem.xlsx") # Change the path to where you want it to be

**_After getting the IDs, I will use the IDs to access the corresponding players' profiles. 

**Note**: for some columns, I was not able to scrap by using Regex. Consequently, results for "age" and "position" were not accurate and my scrapping was interruptted several times after reading hundreds of players. 

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time

df = pd.read_excel(r"C:\Users\qiuwk\Google Drive\Projects\PES.xlsx") # Adjust the path
df_2 = pd.read_excel(r"C:\Users\qiuwk\Google Drive\Projects\PES_tem.xlsx") # Adjust the path
id_list = list(df_2['ID'])

error_list = []

count = 0
base_url = 'http://pesdb.net/pes2020'

for i in id_list:
    url = base_url + i[1:]
    print(url)
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features = 'lxml')
    
    result = soup.find_all("td")
    data = [str(result[1])[4:-5],str(result[7])[4:-5],str(result[8])[4:-5],str(result[9])[4:-5]]
    
    raw_position = str(result[12])
    test = re.search(r">(\w+)</div",raw_position)
    
    # the code was interrupted due a dismatch of position, so I did a test here and try to adjust the data later
    
    if test is None:
        position = 0
    else:    
        re.search(r">(\w+)</div",raw_position).group(1)
        
    data.append(position)
    
    for j in range(25):
        ID = 'a'+str(j)
        AClass_result = str(soup.find(id=ID))
        data.append(re.search(r">(\d+)<"",AClass_result).group(1)) 
        
    for k in range(42,46):
        number = str(result[k])
        data.append(number[-6])
                              
    last_one = str(soup.find(id='a25'))
    data.append(re.search(r'>(\d+)<',last_one).group(1))
    
    df.loc[id_list.index(i)+1] = data
    print(data)
    
    count += 1
    if count == 10:
        time.sleep(115)
        count = 0


**Here, I export the data separatly since I want to make sure my results are mostly accurate.**

In [65]:
# I hid the second line first to check df before writing it into a new spreadsheet
df 

df.to_excel(r"C:\Users\qiuwk\Google Drive\Projects\PES_1.xlsx")

**I realized that I can go to the pages where I retrived IDs to get the positions. So I did that to fill out the "zeros" in my spreadsheet**

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time

position_list = []
count = 0
for i in range(1,471):
    html = urlopen('http://pesdb.net/pes2020/?page=' + str(i)).read()
    soup = BeautifulSoup(html, features = 'lxml')
    raw = soup.find_all(class_ = re.compile("pos[A-Z][A-Z]"))
    for j in raw:
        position = re.search(r'>([A-Z][A-Z][A-Z]?)</div',str(j)).group(1)
        position_list.append(position)
    count +=1
    if count == 10:
        time.sleep(115)
        count = 0
    print(i)
    
df_position = pd.DataFrame(position_list)
df_position.to_excel(r"C:\Users\qiuwk\Google Drive\Projects\position.xlsx") # Change the path to where you want it to be

**I realized there are some mismatches of "height", "weight", "age." So, I firstly used filter function to move the "weight" and "age" column to the correct positions. Then, I run the following codes to scrap the height again.**

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time

height_list = []
count = 0
for i in range (1,471):
    html = urlopen('http://pesdb.net/pes2020/?page=' + str(i)).read()
    soup = BeautifulSoup(html, features = 'lxml')
    raw = soup.find_all("td")
    for j in raw:
        result = re.search("<td>[0-9][0-9][0-9]</td>",str(j))
        if result is not None:
            result_int = int(re.search("^<td>([0-9][0-9][0-9])</td>$",str(j)).group(1))
            if result_int > 130:
                height_list.append(re.search("^<td>([0-9][0-9][0-9])</td>$",str(j)).group(1))
   
    count += 1
    if count == 10:
        time.sleep(115)
        count = 0
        
    print(i)
    
df_height = pd.DataFrame(height_list)
df_height.to_excel(r"C:\Users\qiuwk\Google Drive\Projects\height.xlsx")

**Luckily, as I checked my data, I could did some manual work to fix them.** (It is stupid...but forgive me as a beginner...)

## Conclusion & Mistakes

    From knowing nothing, I spent approximately two weeks to complete the data collection. This database is very organized, but there is still some works to modify the scraped data. My next step will be to perform analysis, majorly multi-regression analysis, to figure out how PES evaluates a player given all the breakdowns. 
    
    For improvement, I would like to that 